# Algorytmy Tekstowe - lab 4
## Odległość edycyjna i najdłuższy wspólny podciąg

### Imports

In [1]:
import numpy as np
import copy
import xx_ent_wiki_sm
import random
import bisect

## Algorytm obliczania odległości edycyjnej

In [2]:
ADD = 0
DELETE = 1
CHANGE = 2
N_CHANGE = 3

def delta1(a,b):
    if a == b:
        return 0
    else:
        return 1

def get_edit_trace(x, y, edit_table, delta=delta1):
    i_x = len(x)
    i_y = len(y)
    ops_vector = []
    while i_x > 0 or i_y > 0:
        computed_delta = delta(x[i_x-1], y[i_y-1])
        possibilities = [
            ((i_x-1, i_y), edit_table[i_x-1, i_y] + 1),
            ((i_x, i_y-1), edit_table[i_x, i_y-1] + 1),
            ((i_x-1, i_y-1), edit_table[i_x-1, i_y-1] + computed_delta)
        ]
        possibilities = list(
            map(
                lambda t: t[0],
                filter(
                    lambda t: t[1] == edit_table[i_x, i_y], 
                    possibilities
                )
            )
        )
        chosen_route = possibilities[0]
        if chosen_route == (i_x-1, i_y):
            ops_vector.insert(0, DELETE)
        elif chosen_route == (i_x, i_y-1):
            ops_vector.insert(0, ADD)
        elif computed_delta == 1:
            ops_vector.insert(0, CHANGE)
        else:
            ops_vector.insert(0, N_CHANGE)
            
        (i_x, i_y) = chosen_route
    return ops_vector
    
def edit_distance(x, y, delta=delta1):
    len_x, len_y = len(x), len(y)
    
    edit_table = np.empty((len_x + 1, len_y + 1), dtype=np.int64)

    edit_table[0,:] = np.linspace(0, len_y, len_y+1, dtype = np.int64)
    edit_table[:,0]= np.linspace(0, len_x, len_x+1, dtype = np.int64)
    
    for i, x_l in enumerate(x, 1):
        for j, y_l in enumerate(y, 1):
            delta_computed = delta(x_l, y_l)
            possibilities = [
                    edit_table[i, j-1] + 1, 
                    edit_table[i-1, j] + 1,
                    edit_table[i-1, j-1] + delta_computed
                  ]
            edit_table[i, j] = min(possibilities)

    return edit_table[len_x, len_y], get_edit_trace(x, y, edit_table, delta)

In [3]:
def show_edit_operations(x, y, ops_vector):
    x_index = 0
    y_index = 0
    for operation in ops_vector:
        if operation == ADD:
            print(x[:x_index] + '*' + y[y_index] + '*' + x[x_index:] + " -- inserting")
            x = x[:x_index] + y[y_index] + x[x_index:]
            x_index += 1
            y_index += 1
        elif operation == DELETE:
            print(x[:x_index] + '|' + x[x_index] + '|' + x[x_index + 1:] + " -- deleting")
            x = x[:x_index] + x[x_index + 1:]
        elif operation == CHANGE:
            print(x[:x_index] + '[' + x[x_index] + "->" + y[y_index]+ ']' + (x[x_index + 1:] if x_index + 1 < len(x) else "") + " -- changing")
            x = x[:x_index] + y[y_index] + x[x_index + 1:]
            x_index += 1
            y_index += 1
        else:
            x_index += 1
            y_index += 1

In [4]:
ex1 = ("los", "kloc")
ex2 = ("Łódź", "Lodz")
ex3 = ("kwintesencja", "quintessence")
ex4 = ("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

ex_list = [ex1, ex2, ex3, ex4]

In [5]:
for a, b in ex_list:
    print("___________________________")
    edit_length, ops_vec = edit_distance(a, b, delta1)
    print(f"Edit length for \n\t{a} \n\t{b} \nis {edit_length}")
    print("Editing operations:")
    show_edit_operations(a, b, ops_vec)

___________________________
Edit length for 
	los 
	kloc 
is 2
Editing operations:
*k*los -- inserting
klo[s->c] -- changing
___________________________
Edit length for 
	Łódź 
	Lodz 
is 3
Editing operations:
[Ł->L]ódź -- changing
L[ó->o]dź -- changing
Lod[ź->z] -- changing
___________________________
Edit length for 
	kwintesencja 
	quintessence 
is 5
Editing operations:
[k->q]wintesencja -- changing
q[w->u]intesencja -- changing
quintes*s*encja -- inserting
quintessenc[j->e]a -- changing
quintessence|a| -- deleting
___________________________
Edit length for 
	ATGAATCTTACCGCCTCG 
	ATGAGGCTCTGGCCCCTG 
is 7
Editing operations:
ATGA[A->G]TCTTACCGCCTCG -- changing
ATGAG[T->G]CTTACCGCCTCG -- changing
ATGAGGCT*C*TACCGCCTCG -- inserting
ATGAGGCTCT[A->G]CCGCCTCG -- changing
ATGAGGCTCTG*G*CCGCCTCG -- inserting
ATGAGGCTCTGGCC|G|CCTCG -- deleting
ATGAGGCTCTGGCCCCT|C|G -- deleting


### Narzędzie do usuwania losowych tokenów z pliku

In [6]:
def remove_rand_tokens(text, percentage=3):
    nlp = xx_ent_wiki_sm.load()
    tokenizer = nlp.Defaults.create_tokenizer(nlp)

    tokenized = tokenizer(text)

    indexes = [i for i, token in enumerate(tokenized) if token.text != '\n']
    
    random.shuffle(indexes)
    
    to_delete = indexes[:int((percentage/100)*len(indexes))]
    to_delete.sort()
    
    to_del_len = len(to_delete)
    to_del_ind = 0
    result = ""
    
    for i, token in enumerate(tokenized):
        if to_del_ind < to_del_len and i == to_delete[to_del_ind]:
            to_del_ind += 1
            if not result[-1].isspace():
                result += " "
        else:
            result += token.text_with_ws
            
    return result
            
def remove_rand_tokens_from_file(in_filename, out_filename, percentage=3):
    text = None
    with open(in_filename, "r") as in_file:
        text = in_file.read()
    with open(out_filename, "w") as out_file:
        out_file.write(remove_rand_tokens(text, percentage))

In [7]:
src_filename = "romeo-i-julia.txt"
modified1_filename = "romeo.txt"
modified2_filename = "julia.txt"

# remove_rand_tokens_from_file(src_filename, modified1_filename)
# remove_rand_tokens_from_file(src_filename, modified2_filename)

### Liczenie ilości tokenów w plikach

In [8]:
def count_tokens(text):
    nlp = xx_ent_wiki_sm.load()
    tokenizer = nlp.Defaults.create_tokenizer(nlp)
    return len(tokenizer(text))

def count_tokens_in_file(filename):
    with open(filename, 'r') as file:
        return count_tokens(file.read())

In [9]:
print(f"{src_filename} tokens: {count_tokens_in_file(src_filename)}")
print(f"{modified1_filename} tokens: {count_tokens_in_file(modified1_filename)}")
print(f"{modified2_filename} tokens: {count_tokens_in_file(modified2_filename)}")

romeo-i-julia.txt tokens: 32164
romeo.txt tokens: 31248
julia.txt tokens: 31248


## Najdłuższy wspólny podciąg

In [10]:
def get_lcs_from_table(x, y, lcs_table):
    ind_x, ind_y = len(x), len(y)
    result = []
    while lcs_table[ind_x, ind_y] != 0:
        next_ind_x, next_ind_y = max([(ind_x-1, ind_y), (ind_x, ind_y-1)], key=lambda t: lcs_table[t[0], t[1]])
        if lcs_table[ind_x, ind_y] == lcs_table[next_ind_x, next_ind_y]:
            ind_x, ind_y = next_ind_x, next_ind_y
        else:
            result = [x[ind_x - 1]] + result
            ind_x -= 1
            ind_y -= 1
    return result
    
def lcs(x, y):
    len_x, len_y = len(x), len(y)
    lcs_table = np.zeros((len_x + 1, len_y + 1), dtype=np.int64)

    for i, x_letter in enumerate(x):
        k = i + 1
        for j, y_letter in enumerate(y):
            l = j + 1
            if x_letter == y_letter:
                lcs_table[k, l] = lcs_table[k-1, l-1] + 1
            else:
                lcs_table[k, l] = max(lcs_table[k, l-1], lcs_table[k-1, l])

    return get_lcs_from_table(x, y, lcs_table)

def lcs_len(x,y):
    # Getting LCS length does not require memorizing lcs_table
    # So another approach is chosen to save memory
    
    ranges = []
    positions = {}
    ranges.append(len(y))
    y_letters = list(y)
    
    for j, l in enumerate(y_letters):
        if l not in positions:
            positions[l] = []
        positions[l] = [j] + positions[l] 
        
    for x_letter in x:
        if x_letter in positions:
            for p in positions[x_letter]:
                k = bisect.bisect(ranges,p)
                if k == bisect.bisect(ranges,p-1):
                    if k < len(ranges)-1:
                        ranges[k] = p
                    else:
                        ranges[k:k]=[p]
    return len(ranges)-1

### Najdłuższy wspólny podciąg tokenów w plikach
Na przykładzie dwóch plików z treścią dramatu "Romeo i Julia" Williama Shakespeare'a z losowo usuniętymi tokenami (3% ze wszytkich tokenów zostało usuniętych) 

In [11]:
def lcs_len_of_tokens(text1, text2):
    nlp = xx_ent_wiki_sm.load()
    tokenizer = nlp.Defaults.create_tokenizer(nlp)
    
    tokenized1 = tokenizer(text1)
    tokenized2 = tokenizer(text2)
    return lcs_len(list(map(lambda t: t.text, tokenized1)), list(map(lambda t: t.text, tokenized2)))

def lcs_len_of_tokens_in_files(filename1, filename2):
    with open(filename1, "r") as file1:
        with open(filename2, "r") as file2:
            return lcs_len_of_tokens(file1.read(), file2.read())

In [12]:
found_lcs_len = lcs_len_of_tokens_in_files(modified1_filename, modified2_filename)

print(f"Tokens longest common subsequence length: {found_lcs_len}")

Tokens longest common subsequence length: 30329


### Narzędzie na wzór 'diff'

In [13]:
def lines_numbers_interval(first_index, lines_cnt):
    if lines_cnt > 1:
        return f"{first_index},{first_index + lines_cnt - 1}"
    else:
        return str(first_index)

def diff(filename1, filename2):
    lines1 = None
    lines2 = None
    with open(filename1, 'r') as file:
        lines1 = list(file.readlines())
    
    with open(filename2, 'r') as file:
        lines2 = list(file.readlines())
        
    common_subseq = lcs(lines1, lines2)
    ind1 = 0
    ind2 = 0
    for common_line in common_subseq:
        unique_lines1 = []
        unique_lines2 = []
        first_ind1 = ind1 + 1
        first_ind2 = ind2 + 1
        while lines1[ind1] != common_line:
            unique_lines1.append(lines1[ind1][:-1])
            ind1 += 1

        while lines2[ind2] != common_line:
            unique_lines2.append(lines2[ind2][:-1])
            ind2 += 1

        ind1 += 1
        ind2 += 1
        
        if unique_lines1 and unique_lines2:
            mode = 'c'
        elif unique_lines1:
            mode = 'd'
        elif unique_lines2:
            mode = 'a'
        else:
            continue

        print(f"{lines_numbers_interval(first_ind1, len(unique_lines1))}{mode}{lines_numbers_interval(first_ind2, len(unique_lines2))}")
        for line in unique_lines1:
            print(f"< {line}")
            
        if mode == 'c':
            print("---")
            
        for line in unique_lines2:
            print(f"> {line}")

### Działanie diff
Także na przykładzie dwóch wersji "Romea i Julii" z usuniętymi tokenami

In [14]:
diff(modified1_filename, modified2_filename)

1c1
< William 
---
> William Shakespeare
12,14c12,14
<  * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia
<  * MONTEKI, — naczelnicy dwóch domów nieprzyjaznych sobie
<  * STARZEC — stryjeczny Kapuleta
---
>  * PARYS — młody Weroneńczyk szlachetnego rodu krewny księcia
>  * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
>  * STARZEC — stryjeczny brat Kapuleta
16,17c16
<  * MERKUCJO — krewny księcia
<  * BENWOLIO — synowiec 
---
>  * — krewny księcia * BENWOLIO — synowiec Montekiego
21,22c20
<  * BALTAZAR — służący Romea
<  * SAMSON, GRZEGORZ — słudzy Kapuleta
---
>  * BALTAZAR — służący Romea * SAMSON, GRZEGORZ — słudzy Kapuleta
29c27
<  PANI MONTEKI — małżonka Montekiego
---
>  * PANI MONTEKI — małżonka Montekiego
33c31
<  * Obywatele weroneńscy, różne osoby obojej, się do przyjaciół obu domów, maski, straż wojskowa i inne osoby.
---
>  * Obywatele weroneńscy, różne osoby płci obojej, się do przyjaciół obu domów, maski, straż wojskowa i inne osoby.
47,48c45,

---
> Jedno mieć tylko, jedno biedne dziecko 
5845c5838,5840
< MARTA O smutny, smutny dniu! o dniu żałosny!
---
> MARTA
> 
> O smutny, smutny dniu! o dniu żałosny!
5848,5850c5843,5845
< O dniu! o smutny dniu! O dniu żałosny!
< Nie nigdy jeszcze dnia takiego.
< O! stokroć smutny dniu, stokroć żałosny! PARYS
---
> O dniu! o smutny ! O dniu żałosny!
> Nie było nigdy jeszcze dnia takiego.
> O! stokroć smutny , stokroć żałosny 
5852a5847,5849
> 
> PARYS
> 
5855,5856c5853,5854
< O Julio! luba! życie! już nie życie.
< Nie mniej luba i po śmierci!
---
> Julio! luba! życie! nie życie.
> Nie mniej jednakże luba i po śmierci!
5862c5860
< Po cóż ci, co było tak tyrańsko
---
> Po cóż ci, po co było tak tyrańsko
5864,5867c5862,5865
< O moje dziecko! raczej duszo moja,
< Nie moje dziecko, bo dziecko jest trupem;
< I wraz z nim cała pociech mych ostoja 
< Cały wdzięk życia stał się śmierci łupem!
---
> O moje dziecko! raczej duszo moja 
> Nie dziecko, bo dziecko jest trupem;
> I wraz z nim cała pociec